# RAG Q&A LLM Chatbot for [Confluence Space](https://templates.atlassian.net/wiki/spaces/SWPRJ) and Github Repositories ([Full-Stack App](https://crowd-wire.github.io/ProjectDocumentation/), [SQL Tables](https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples))

# Dependencies

In [1]:
import os
import re
import locale
import openai
from dotenv import load_dotenv
# langchain
from langchain.document_loaders import ConfluenceLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
# llama_index
from llama_index import Document
from llama_index import ServiceContext, LLMPredictor, LangchainEmbedding
from llama_index import Prompt, VectorStoreIndex
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo

locale.getpreferredencoding = lambda: "UTF-8"

# Set API Keys

In [2]:
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

# hf access token
hf_token = os.getenv("HF_API_KEY")

# Loading the Documents

In [3]:
headers_to_split_on = [
    ("#", "title"),
    ("##", "header"),
    ("###", "header"),
    ("####", "header"),
    ("#####", "header"),
    ("######", "header"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=200)

def clean_text(text):
    # remove words with more than 100 characters
    text = re.sub(r'\b\w{101,}\b', ' ', text)
    # remove extra white spaces
    text = re.sub(r' +', ' ', text)
    return text

# final documents array
documents = []

## Confluence loader

In [4]:
# Confluence Loader (will transform later the web pages to markdown format)
confluence_loader = ConfluenceLoader(
    url="https://templates.atlassian.net/wiki"
)

if callable(confluence_loader.load):
  for doc in confluence_loader.load(
      space_key="SWPRJ",
      include_attachments=False,
      keep_markdown_format=True
    ):
    link = doc.metadata["source"]
    source = link.split("/")[-1]
    title = doc.metadata["title"]

    doc.page_content = clean_text(doc.page_content)
    docs = markdown_splitter.split_text(doc.page_content)
    docs = text_splitter.split_documents(docs)

    for doc in docs:
      doc.metadata["link"] = link
      doc.metadata["source"] = source
      doc.metadata["title"] = title
      if "header" not in doc.metadata:
        doc.metadata["header"] = title
      documents.append(Document(
          text = doc.page_content,
          metadata=doc.metadata,
          excluded_llm_metadata_keys = ["link"],
          excluded_embed_metadata_keys = ["link"]
        ))

In [5]:
len(documents)

68

## Repository Loader

In [6]:
# Repository Loader (Markdowns only for now)
glob_pattern = "*.md"
repo_paths = [
  "repos/ProjectDocumentation-master",
  "repos/SQL_Examples"
]
repo_links = [
  "https://github.com/Crowd-Wire/ProjectDocumentation/tree/master/",
  "https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples/"
]

repo_i = 0
for repo_path in repo_paths:
  try:
    repo_loader = DirectoryLoader(
      repo_path,
      glob=glob_pattern,
      show_progress=True,
      recursive=True,
      loader_cls=TextLoader,
      loader_kwargs={"encoding": "utf8"})
  except Exception as e:
    print(f"Error loading files with pattern '{glob_pattern}': {e}")
    break

  if callable(repo_loader.load):
    for doc in repo_loader.load():
      relative_path = os.path.relpath(doc.metadata['source'], repo_path)
      link = repo_links[repo_i] + "/" + relative_path

      doc.page_content = clean_text(doc.page_content)
      docs = markdown_splitter.split_text(doc.page_content)
      docs = text_splitter.split_documents(docs)

      for doc in docs:
        if "title" in doc.metadata:
          title = doc.metadata["title"]
          break

      for doc in docs:
        doc.metadata['link'] = link
        doc.metadata['source'] = relative_path
        doc.metadata['title'] = title
        if "header" not in doc.metadata:
          doc.metadata["header"] = title
        documents.append(Document(
          text = doc.page_content,
          metadata=doc.metadata,
          excluded_llm_metadata_keys = ["link"],
          excluded_embed_metadata_keys = ["link"]
        ))
    repo_i += 1

100%|██████████| 4/4 [00:00<00:00, 3993.62it/s]


In [7]:
len(documents)

112

## Gathered Documents

In [8]:
len(documents)

112

In [9]:
def pretty_print_docs(docs):
  print(f"\n{'-' * 100}\n".join([f"Document {i+1} | \
Title: {d.metadata['title']} | \
Header: {d.metadata['header']} | \
Source: {d.metadata['source']} | \
Link: {d.metadata['link']}\n\
Content:\n{d.text}" for i, d in enumerate(docs)]))

def pretty_print_answer(answer):
  print(f"Answer: {answer}\n\n")

def pretty_print_result(result):
  print(f"Question: {result['query']}\n\n")
  print(f"Answer: {result['result']}\n\n")
  print("-" * 100)
  print("Documents:\n")
  pretty_print_docs(result["source_documents"])

def pretty_print_response(question):
  response = query_engine.query(question)
  print(f"Question: {question}\n\n")
  print(f"Answer: {response.response}\n\n")
  print("-" * 100)
  print("Documents used:\n")
  pretty_print_docs(response.source_nodes)

In [10]:
pretty_print_docs(documents)

Document 1 | Title: Demo Project Home | Header: Demo Project Home | Source: overview | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/overview
Content:
#DDDDDD#f5f5f5#CCCCCC1none## Mobile App Project Shortcuts  
---
----------------------------------------------------------------------------------------------------
Document 2 | Title: Demo Project Home | Header: Demo Project Home | Source: overview | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/overview
Content:
To create a 5-star mobile app that our users adore.
----------------------------------------------------------------------------------------------------
Document 3 | Title: Demo Project Home | Header: Demo Project Home | Source: overview | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/overview
Content:
INLINE* Reach 50,000,000 downloads by March 31st.
* 30% of new user evaluations begin on mobile.
* Have 95% positive user reviews.
--------------------------------------------------------------------

# Creating Embeddings

In [11]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"}
  )
)

llm_predictor = LLMPredictor(
  llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
)

service_context = ServiceContext.from_defaults(
  llm_predictor=llm_predictor,
  embed_model=embed_model,
)

C:\Users\Chico\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Preparing the Retrieval QA LLM with RAG

In [12]:
# This isnt used, only if VectorIndexAutoRetriever was used
vector_store_info = VectorStoreInfo(
    content_info="Documents containing information about projects and SQL tables and columns",
    metadata_info=[
      MetadataInfo(
        name="title",
        description="Title of the document",
        type="str",
      ),
      MetadataInfo(
        name="header",
        description="Subtitle of the document",
        type="str",
      ),
      MetadataInfo(
        name="source",
        description="Filename and location of the source file",
        type="str",
      ),
      MetadataInfo(
        name="link",
        description="Link of the source file",
        type="str",
      ),
    ],
)

In [13]:
similarity_top_k = 6
mmr_threshold = 0.7

# define a prompt template
text_qa_template = Prompt("""You are a QA Bot. Based on the documents answer the question at the end. Don't make up an answer if you don't know the answer.

{context_str}

Question: {query_str}

Answer: """)


# build index
index = VectorStoreIndex.from_documents(
  documents,
  service_context=service_context,
  similarity_top_k=similarity_top_k,
  vector_store_query_mode="mmr",
  vector_store_kwargs={"mmr_threshold": mmr_threshold}
)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    # vector_store_info=vector_store_info,
    similarity_top_k=similarity_top_k,
    vector_store_query_mode="mmr",
    vector_store_kwargs={"mmr_threshold": mmr_threshold},
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
  text_qa_template=text_qa_template,
  service_context=service_context,
  response_mode="compact"
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# Question Answering

## [Confluence Space](https://templates.atlassian.net/wiki/spaces/SWPRJ)

In [14]:
pretty_print_response("What are the confluence user story dashboard requirements?")

Question: What are the confluence user story dashboard requirements?


Answer: The Confluence user story dashboard requirements are as follows:

1. Activity feed: Users want to see the most recent activity related to their work so that they can efficiently triage their work for the day. This includes ordering pages chronologically with the most recent updates first and showing a different color link when a page has been visited already. This requirement is a must-have.

2. Comment excerpt: Users want to see comments in the activity feed so that they can see what someone has said without going all the way to a page. The maximum length of a comment is 280 characters, and it should be truncated with an ellipsis. This requirement is a nice-to-have.

3. Save for later: Users want to be able to save a page for later so that they can quickly go back to it on their desktop. This requirement should include a saved/not saved status icon. This requirement is a nice-to-have.

4. Share: Users want 

In [15]:
pretty_print_response("What is and what is the summary of the confluence 2016-05-23 Retrospective")

Question: What is and what is the summary of the confluence 2016-05-23 Retrospective


Answer: The summary of the confluence 2016-05-23 Retrospective is that 75% of issues were resolved and deployed, the new QA process helped eliminate last-minute blocking bugs, designs were finalized and ready in advance of starting the sprint, feedback on pull requests was added within a day, no critical bugs were introduced. They should have done better in resolving build issues and considering the time for back-end development in future estimations.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: 2016-05-23 Retrospective | Header: Retrospective | Source: 2016-05-23+Retrospective | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/33560/2016-05-23+Retrospective
Content:
#91c89c#f3f9f4### What did we do well?  
* 75% of issues were resolved and deployed.
* New QA process helped eliminate last-minut

In [16]:
pretty_print_response("What projects does the confluence space has? Give a short description if possible")

Question: What projects does the confluence space has? Give a short description if possible


Answer: Based on the provided documents, the Confluence space has the following projects:

1. Sprint planning: This project allows users to plan their meetings and share notes and actions with their team. It is a blueprint for creating meeting notes.

2. Demo Project Home: This project provides a concise overview of a demo project. It includes pages, comments, and blog posts related to the project.

3. Retrospectives: This project is focused on creating retrospectives. It allows users to create retrospectives and view them. The blueprint used for this project is the file list blueprint.

4. System Technology Architecture: This project includes documentation related to the system technology architecture. It contains an image of the architecture diagram and provides information about CrowdWire - Massive Online Meetings.

5. Decision log: This project is used for recording important project decis

In [17]:
pretty_print_response("How are search results ordered?")

Question: How are search results ordered?


Answer: The search results should be ordered by date.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: How should search results be ordered? | Header: Background | Source: 33510 | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/33510
Content:
The 'Search' function is used to find relevant pages, blogs, files and comments on Confluence. These display the page, blog or file name, the space they're in, and the name of the person who created them. How should these be ordered?
----------------------------------------------------------------------------------------------------
Document 2 | Title: How should search results be ordered? | Header: Decision | Source: 33510 | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/33510
Content:
| Option | Positives | Negatives | Unknowns |
| --- | --- | --- | --- |
| By date | Surfaces the most

In [14]:
pretty_print_response("What is planned for notifications?")

Question: What is planned for notifications?


Answer: Based on the given documents, it is stated in the "2016-06-19 Sprint Planning "Notifications"" document that the goal for this sprint is to implement a working prototype for notifications. Therefore, the plan for notifications is to develop and implement a working prototype.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: 2016-06-19 Sprint Planning "Notifications" | Header: Goals | Source: 33655 | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/33655
Content:
grey1dashed### In this sprint we aim to implement a working prototype for notifications.  
---
----------------------------------------------------------------------------------------------------
Document 2 | Title: What does the notification feed blank state look like? | Header: Background | Source: 33547 | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/335

In [15]:
pretty_print_response("What are some questions that the the confluence space answers?")

Question: What are some questions that the the confluence space answers?


Answer: Some questions that the Confluence space answers include:
- What is this site?
- What is the purpose of this site?
- What features does this site display?
- What is the decision log?
- What is CrowdWire and its milestones?
- What is included in the Retrospectives section?
- Should blog posts be included in the notification feed?
- What are the positives, negatives, and unknowns of including blogs in the notification feed?
- Should there be a separate feed for blogs?
- What is the Sprint planning section about?


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: What is this site ? | Header: What is this site ? | Source: 1972475 | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/1972475
Content:
We've created this site to enable you to see how a Confluence site looks like.  
You can visit this site just as

## [Full-Stack App GH Repo](https://crowd-wire.github.io/ProjectDocumentation/)

In [16]:
pretty_print_response("What is CrowdWire?")

Question: What is CrowdWire?


Answer: CrowdWire is a project for massive online meetings that includes features such as world editing, full integration of micro-services, proximity video and voice chatting with high availability, CI/CD, and a file exchange feature. It utilizes technologies such as FastAPI, ReactJS, Kubernetes, and Mediasoup for media server communications.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: CrowdWire - Massive Online Meetings | Header: 3rd Milestone | Source: README.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//README.md
Content:
- World-Editor
- Full Integration of each micro-service
- Proximity video and voice chatting with high availability
- CI/CD
- File Exchange feature
----------------------------------------------------------------------------------------------------
Document 2 | Title: Welcome to CrowdWire | Header: Project orga

In [17]:
pretty_print_response("What is the structure of CrowdWire kubernets charts?")

Question: What is the structure of CrowdWire kubernets charts?


Answer: The structure of CrowdWire Kubernetes charts is as follows:

- Chart.lock
- Chart.yaml
- subcharts
  - api
  - frontend
  - mediaserver
  - postgresql
  - rabbitmq
  - redis
- templates
  - _helpers.tpl
  - ingress.yaml
- values.yaml


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: CrowdWire - Massive Online Meetings | Header: Run our Helm Charts | Source: docs\deployment.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs\deployment.md
Content:
To manage the packaging of the Kubernetes resources, we are using [Helm]( https://helm.sh/). If you want to make use of our charts locally, do the following steps. Before installing this chart please check the previous deployment notes.  
```
$cd charts

$helm dep build

$helm install crowdwire .
```  
To observe the initialization of all Pods, use the fol

In [18]:
pretty_print_response("What are the technologies stack used in CrowdWire?")

Question: What are the technologies stack used in CrowdWire?


Answer: The technologies stack used in CrowdWire includes FastAPI for the backend Rest API, ReactJS for the project website, Mediasoup for the media server, and Kubernetes for deployment.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: CrowdWire - Massive Online Meetings | Header: 1st Milestone | Source: README.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//README.md
Content:
- Set up docs platforms tools
- Develop a prototype
- Research technologies and define the system architecture
- Start testing and developing shortly each area/framework individually
----------------------------------------------------------------------------------------------------
Document 2 | Title: Welcome to CrowdWire | Header: Project organization | Source: docs\index.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation

In [19]:
pretty_print_response("What is the Crowdwire media server framework?")

Question: What is the Crowdwire media server framework?


Answer: The Crowdwire media server framework is Mediasoup.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: Welcome to CrowdWire | Header: Overview | Source: docs\media_server.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs\media_server.md
Content:
[Mediasoup](https://mediasoup.org/) has an SFU (Selective Forwarding Unit) topology, where the server routes media around between participants while balancing its limitations with the media inputs it receives. With this approach, participants send their media to the server and receive others' media in separate streams, one each.  
For more details of how mediasoup works and how we interacted with it within the browser, check out their documentation:  
- [Server Side Library](https://mediasoup.org/documentation/v3/mediasoup/api/)
- [Client Side Library](https://med

In [20]:
pretty_print_response("What is the Mediasoup website link?")

Question: What is the Mediasoup website link?


Answer: The Mediasoup website link is [https://mediasoup.org/](https://mediasoup.org/).


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: Welcome to CrowdWire | Header: Overview | Source: docs\media_server.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs\media_server.md
Content:
[Mediasoup](https://mediasoup.org/) has an SFU (Selective Forwarding Unit) topology, where the server routes media around between participants while balancing its limitations with the media inputs it receives. With this approach, participants send their media to the server and receive others' media in separate streams, one each.  
For more details of how mediasoup works and how we interacted with it within the browser, check out their documentation:  
- [Server Side Library](https://mediasoup.org/documentation/v3/mediasoup/api/)
- [Client Side L

In [21]:
pretty_print_response("How are messages traded between client, backend and the media server?")

Question: How are messages traded between client, backend and the media server?


Answer: Messages are traded between the client, backend, and the media server using RabbitMQ. RabbitMQ is used for communication between the MediaSoup server and the FastAPI backend. The API replicas created by Kubernetes also use RabbitMQ to share information. The different protocols used in the RabbitMQ channels are defined in the `RabbitProtocol` class.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: Welcome to CrowdWire | Header: Overview | Source: docs\media_server.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs\media_server.md
Content:
[Mediasoup](https://mediasoup.org/) has an SFU (Selective Forwarding Unit) topology, where the server routes media around between participants while balancing its limitations with the media inputs it receives. With this approach, participants sen

## [SQL Tables GH Repo](https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples)

In [22]:
pretty_print_response("What is the column joined?")

Question: What is the column joined?


Answer: The column "joined" refers to the date when the employee joined the company.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: Welcome to CrowdWire | Header: DB Modelation | Source: docs\REST_API.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs\REST_API.md
Content:
To model our database we used the following diagram:
----------------------------------------------------------------------------------------------------
Document 2 | Title: data.users | Header: data.users | Source: tables.md | Link: https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples//tables.md
Content:
Employees who have software licenses.  
| ID | Name | Type(Length) | Nullable | Default | Example | Comment |
| ---- | ------ | -------------- | ---------- | --------- | ---------------------------- | ----------------------------------------- |
| 

In [23]:
pretty_print_response("What is the table users?")

Question: What is the table users?


Answer: The table "users" is a table that stores information about employees who have software licenses. It includes columns such as ID, Name, Email, Age, and Joined.


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: Welcome to CrowdWire | Header: DB Modelation | Source: docs\REST_API.md | Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs\REST_API.md
Content:
To model our database we used the following diagram:
----------------------------------------------------------------------------------------------------
Document 2 | Title: How-to open access rights for our customers | Header: How-to open access rights for our customers | Source: How-to+open+access+rights+for+our+customers | Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/1975532/How-to+open+access+rights+for+our+customers
Content:
Add the steps involved:  
1. Open t

In [24]:
pretty_print_response("What is the column for the date the software was installed?")

Question: What is the column for the date the software was installed?


Answer: The column for the date the software was installed is "installed".


----------------------------------------------------------------------------------------------------
Documents used:

Document 1 | Title: data.software | Header: data.software | Source: data\software.md | Link: https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples//data\software.md
Content:
Software bought for or by the company  
| ID | Name | Type(Length) | Nullable | Default | Example | Comment |
| ---- | --------- | -------------- | ---------- | --------- | ---------------- | ------------------------------------------ |
| 1 | id | int(4) | false | 0 | 5 | Primary Key, technical |
| 2 | name | nvarchar(800) | true | 0 | “Office 2016” | Name of the software |
| 3 | creator | nvarchar(800) | true | 0 | “Microsoft” | Creator behind the software |
| 4 | website | nvarchar(800) | true | 0 | “www.office.com” | Website of the softwar